# Cosyne 2020 NWB:N Tutorial - Extracellular Electrophysiology

## Introduction
In this tutorial, we will create fake data for a hypothetical extracellular electrophysiology experiment with a freely moving animal. The types of data we will convert are:
- Animal position
- LFP
- Spike times
- Trials
- Subject (species, strain, age, etc.) 

## Installing PyNWB
If you are in the tutorial using DANDI Hub, PyNWB is already installed. 
If participating from your own machine, install PyNWB using pip or conda:
- `pip install pynwb`
- `conda install -c conda-forge pynwb`

## Set up the NWB file
An NWB file represents a single session of an experiment. Each file must have a session description, identifier, and session start time. Create a new `NWBFile` object with those and additional metadata. For all PyNWB constructors, we recommend using keyword arguments.

In [1]:
from pynwb import NWBFile
from datetime import datetime
from dateutil import tz

start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz('US/Pacific'))

nwb = NWBFile(session_description='Mouse exploring an open field',
              identifier='Mouse5_Day3',
              session_start_time=start_time,
              session_id='session_1234',                                # optional
              experimenter='My Name',                                   # optional
              lab='My Lab Name',                                        # optional
              institution='University of My Institution',               # optional
              related_publications='DOI:10.1016/j.neuron.2016.12.011')  # optional
nwb

root pynwb.file.NWBFile at 0x4340444112
Fields:
  experimenter: ['My Name']
  file_create_date: [datetime.datetime(2020, 2, 25, 16, 26, 37, 813540, tzinfo=tzlocal())]
  identifier: Mouse5_Day3
  institution: University of My Institution
  lab: My Lab Name
  related_publications: ['DOI:10.1016/j.neuron.2016.12.011']
  session_description: Mouse exploring an open field
  session_id: session_1234
  session_start_time: 2018-04-25 02:30:03-07:00
  timestamps_reference_time: 2018-04-25 02:30:03-07:00

## Subject information
Create a `Subject` object to store information about the experimental subject, such as age, species, genotype, sex, and a freeform description. And set `nwb.subject` to the `Subject` object.


<img src="images/subject_diagram.png" width="600">

In [2]:
from pynwb.file import Subject

nwb.subject = Subject(age='9 months', 
                      description='mouse 5',
                      species='Mus musculus', 
                      sex='M')

## Position
The `Position` object is a special type of object called a `MultiContainerInterface`. It holds one or more `SpatialSeries` objects. 

<img src="images/spatial_series.png" width="600">

## SpatialSeries
`SpatialSeries` is a subclass of `TimeSeries`. `TimeSeries` is a common base class for measurements sampled over time, and provides fields for data and time (regularly or irregularly sampled).

<img src="images/position.png" width="800">

Create a `SpatialSeries` object named `'position'` with some fake data.

In [3]:
import numpy as np
from pynwb.behavior import SpatialSeries, Position

position_data = np.array([np.linspace(0, 10, 100),
                          np.linspace(1, 8, 100)]).T
spatial_series_object = SpatialSeries(
    name='position', 
    data=position_data,
    reference_frame='unknown',
    timestamps=np.linspace(0, 100) / 200)

Then, create a `Position` object which contains the `SpatialSeries` object you just created.

In [4]:
pos_obj = Position(spatial_series=spatial_series_object)

Finally, create a processing module named `'behavior'` in the NWB file and add the `Position` object to the processing module.

In [5]:
behavior_module = nwb.create_processing_module(
    name='behavior',
    description='processed behavioral data')

behavior_module.add(pos_obj)

Position pynwb.behavior.Position at 0x4637807056
Fields:
  spatial_series: {
    position <class 'pynwb.behavior.SpatialSeries'>
  }

## Write to file

In [6]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('ecephys_tutorial.nwb', 'w') as io:
    io.write(nwb)

## Trials

<img src="images/trials.png" width="500">

`DynamicTable` objects are used to store tabular metadata throughout NWB, including electrodes and sorted units. They offer flexibility for tabular data by allowing required columns, optional columns, and custom columns.

Trials are stored in a `TimeIntervals` object which subclasses `DynamicTable`. Here, we are adding a column named `'correct'`, which will be a boolean array.

In [7]:
nwb.add_trial_column(name='correct', description='whether the trial was correct')
nwb.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwb.add_trial(start_time=6.0, stop_time=10.0, correct=False)

## Electrodes table
Extracellular electrodes are stored in a `electrodes` table, which is also a `DynamicTable`. `electrodes` has several required fields: x, y, z, impedence, location, filtering, and electrode_group.

<img src="images/electrodes_table.png" width="300">

Here, we also demonstrate how to add optional columns to a table by adding the `'label'` column.

In [8]:
nwb.add_electrode_column('label', 'label of electrode')
shank_channels = [4, 3]  # set up 4 shanks with 3 electrodes each

electrode_counter = 0
device = nwb.create_device('array')
for shankn, nelecs in enumerate(shank_channels):
    # create an electrode group for this shank
    electrode_group = nwb.create_electrode_group(
       name='shank{}'.format(shankn),
       description='electrode group for shank {}'.format(shankn),
       device=device,
       location='brain area')
    # add electrodes to the electrode table
    for ielec in range(nelecs):
        nwb.add_electrode(
           x=5.3, y=1.5, z=8.5, imp=np.nan,
           location='unknown', filtering='unknown',
           group=electrode_group,
           label='shank{}elec{}'.format(shankn, ielec))
        electrode_counter += 1

## Links
In the above loop, we create `ElectrodeGroup` objects using `nwb.create_electrode_group`. When we add an electrode, we pass in the `ElectrodeGroup` object for the `'group'` argument. This creates a reference from the `electrodes` table to individual `ElectrodeGroup` objects, one per row.

In order to create our `ElectricalSeries` object, we will also need to create a `DynamicTableRegion` of electrodes. A `DynamicTableRegion` is a type of link that allows you to reference specific rows of a `DynamicTable`.

In [9]:
# create a table region object that refs to a set of rows of the table by index
all_table_region = nwb.create_electrode_table_region(
  list(range(electrode_counter)), 'all electrodes')

## LFP
`LFP` is another `MultiContainerInterface`. It holds one or more `ElectricalSeries` objects, which are `TimeSeries`. Here, we put an `ElectricalSeries` named `'lfp'` in an `LFP` object, in a `ProcessingModule` named `'ecephys'`.
<img src="images/lfp.png" width="800">

In [10]:
from pynwb.ecephys import ElectricalSeries, LFP

lfp_data = np.random.randn(100, 7)
ecephys_module = nwb.create_processing_module(
    name='ecephys',
    description='extracellular electrophysiology data')
ecephys_module.add_data_interface(
LFP(ElectricalSeries(name='lfp', 
                     data=lfp_data, 
                     electrodes=all_table_region, 
                     rate=200.)))

## Spike Times
Spike times are stored in another `DynamicTable` of subtype `Units`. The main `Units` table is at `/units` in the HDF5 file. You can add columns to the `Units` table just like you did for `electrodes`. Here, we generate some random spike data and populate the table. 

In [11]:
firing_rate = 20
n_units = 10
for n_units_per_shank in range(n_units):
    n_spikes = np.random.poisson(lam=20)
    spike_times = np.cumsum(np.random.exponential(1/firing_rate, n_spikes))
    nwb.add_unit(spike_times=spike_times)

## Ragged arrays

Spike times are an example of a ragged array - it's like a matrix, but each row has a different number of elements. We can represent this type of data as an indexed column of the units `DynamicTable`. These indexed columns have two components, the vector data object that holds the data and the vector index object that holds the indices in the vector that indicate the row breaks.

<img src="images/ragged_array.png" width="800">

## Write the file

In [12]:
with NWBHDF5IO('ecephys_tutorial.nwb', 'w') as io:
    io.write(nwb)

## Reading NWB data
Data arrays are read passively from the file. Calling `TimeSeries.data` does not read the data values, but presents an h5py object that can be indexed to read data. Index this array just like a numpy array to read only a specific section of the array, or use the `[:]` operator to read the entire thing.

In [13]:
with NWBHDF5IO('ecephys_tutorial.nwb', 'r') as io:
    nwb2 = io.read()

    print(nwb2.processing['ecephys']['LFP'].electrical_series['lfp'].data[:])

[[ 1.53925501  0.99941293  0.67372754 -0.43786788  0.4196522  -0.34339012
   0.28581234]
 [ 0.58873386  1.00620222 -0.40668764  1.34793934  2.83953084  1.06733399
   0.12490513]
 [ 0.08278097 -0.75361395  1.99405767  1.07492313 -1.01932098 -1.55389097
  -1.09215109]
 [-1.37779701  1.09444952  0.92433075 -1.91374778  1.30519944  0.10842834
  -0.81886982]
 [ 0.23922791 -0.40820977  0.08296503 -1.9405386   1.03185867  0.05904873
   0.85930336]
 [-1.4462802  -1.0636564   0.50527531  0.4231644  -0.46689943 -0.75856309
  -0.3196499 ]
 [-0.71459194 -1.60474016  0.53046018  0.96837994 -0.46829813  0.43416632
   0.60461954]
 [ 2.00375452  2.15199963 -0.69765304 -0.9223856   0.3175274  -1.08378769
  -1.16139303]
 [ 2.54234464  0.08601078  0.19128532 -1.03374071  1.50570989 -2.43755515
  -1.65992781]
 [ 1.61047097  0.02689781  1.56005176  0.85584023  0.34098395  0.00464326
  -0.33802126]
 [-0.44797477 -0.15577661  0.21330471  1.08377165 -0.28898037  0.70833442
   1.38647119]
 [ 0.11313897  0.2673

## Accessing data regions
It is often preferable to read only a portion of the data. To do this, index or slice into the `'data'` property. The following takes elements 0:10 in the first dimension and 0:5 in the second dimension from the LFP data we have written.

Accessing ragged arrays is similar- `nwb2.units['spike_times'][0]` only reads the times from the 0th unit.

In [14]:
io = NWBHDF5IO('ecephys_tutorial.nwb', 'r')
nwb2 = io.read()

print('section of lfp:')
print(nwb2.processing['ecephys']['LFP'].electrical_series['lfp'].data[:10,:5])
print('')
print('spike times from 0th unit:')
print(nwb2.units['spike_times'][0])
io.close()

section of lfp:
[[ 1.53925501  0.99941293  0.67372754 -0.43786788  0.4196522 ]
 [ 0.58873386  1.00620222 -0.40668764  1.34793934  2.83953084]
 [ 0.08278097 -0.75361395  1.99405767  1.07492313 -1.01932098]
 [-1.37779701  1.09444952  0.92433075 -1.91374778  1.30519944]
 [ 0.23922791 -0.40820977  0.08296503 -1.9405386   1.03185867]
 [-1.4462802  -1.0636564   0.50527531  0.4231644  -0.46689943]
 [-0.71459194 -1.60474016  0.53046018  0.96837994 -0.46829813]
 [ 2.00375452  2.15199963 -0.69765304 -0.9223856   0.3175274 ]
 [ 2.54234464  0.08601078  0.19128532 -1.03374071  1.50570989]
 [ 1.61047097  0.02689781  1.56005176  0.85584023  0.34098395]]

spike times from 0th unit:
[0.06774798 0.07053292 0.07247753 0.16520165 0.19273485 0.27233836
 0.27965926 0.40510744 0.41551048 0.43695602 0.57366978 0.59169895
 0.70903061 0.82855832]


# Learn more!

## Python tutorials
### See our tutorials for more details about your data type:
* [Extracellular electrophysiology](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html#sphx-glr-tutorials-domain-ecephys-py)
* [Calcium imaging](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ophys.html#sphx-glr-tutorials-domain-ophys-py)
* [Intracellular electrophysiology](https://pynwb.readthedocs.io/en/stable/tutorials/domain/icephys.html#sphx-glr-tutorials-domain-icephys-py)

### Check out other tutorials that teach advanced NWB topics:
* [Iterative data write](https://pynwb.readthedocs.io/en/stable/tutorials/general/iterative_write.html#sphx-glr-tutorials-general-iterative-write-py)
* [Extensions](https://pynwb.readthedocs.io/en/stable/tutorials/general/extensions.html#sphx-glr-tutorials-general-extensions-py)
* [Advanced HDF5 I/O](https://pynwb.readthedocs.io/en/stable/tutorials/general/advanced_hdf5_io.html#sphx-glr-tutorials-general-advanced-hdf5-io-py)


## MATLAB tutorials
* [Extracellular electrophysiology](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/ecephys.html)
* [Calcium imaging](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/ophys.html)
* [Intracellular electrophysiology](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/icephys.html)
